In [1]:
from manifold_learning.data.data_loader import load_csv_dataset

ModuleNotFoundError: No module named 'manifold_learning.data.dataset_loader'

In [3]:
from manifold_learning.nD_cICA import func1